In [ ]:
from utils.plotting import *

In [ ]:
def model(x, p):
    return x ** (2 * p + 1) / (1 + x ** (2 * p))

pparam = dict(xlabel='Voltage (mV)', ylabel=r'Current ($\mu$A)')

x = np.linspace(0.75, 1.25, 201)

fig, ax = plt.subplots()
for p in [10, 15, 20, 30, 50, 100]:
    ax.plot(x, model(x, p), label=p)
    ax.legend(title='Order')
    ax.autoscale(tight=True)
    ax.set(**pparam)

In [ ]:
titanic = sns.load_dataset('titanic')
sns.boxplot(data=titanic, x="age", y="class")

In [1]:
import colorlog, logging

class Formatter(colorlog.ColoredFormatter):

	last_log = 0
	levels = {
		'DEBUG' : "DBG",
		"INFO" : "INF",
		"WARNING" : "WRN",
		"ERROR" : "ERR",
		"CRITICAL" : "CRIT"
	}

	def format(self, record):
		delta_milliseconds = record.relativeCreated - self.last_log
		self.last_log = int(record.relativeCreated)
		record.relativeCreated = f"+{int(delta_milliseconds)}ms"
		record.levelno = f"[{self.levels[record.levelname]}]"
		return super().format(record)

formatter = Formatter(
    # "%(log_color)s%(asctime)s - %(name)s - %(levelname)s - %(message)s (%(filename)s:%(lineno)d)",
    f'{{log_color}}{{asctime}} {{levelno}} {{name}}{{reset}} -- {{msg}} ({{bold_purple}}{{relativeCreated}}{{reset}})',
    # "%(log_color)s%(levelname)-8s%(reset)s %(blue)s%(message)s"
    datefmt='%H:%M:%S',
	reset=True,
	log_colors={
		'DEBUG':    'bold_cyan',
		'INFO':     'bold_white',
		'WARNING':  'bold_yellow',
		'ERROR':    'bold_red',
	},
	secondary_log_colors={},
	style='{'
)

# create logger with 'spam_application'
logger = logging.getLogger("My_app")
logger.setLevel(logging.DEBUG)

# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(0)

ch.setFormatter(formatter)

logger.addHandler(ch)

In [3]:
logger.debug('test 0')
time.sleep(1)
logger.info('test 1')
logger.warning('test 2')
logger.error('test 3')

18:13:25 [DBG] My_app -- test 0 (+8169ms)
18:13:26 [INF] My_app -- test 1 (+1002ms)
18:13:26 [WRN] My_app -- test 2 (+2ms)
18:13:26 [ERR] My_app -- test 3 (+1ms)


In [ ]:
logger.debug('test')
logger.info('test 1')
logger.warning('test 2')
logger.error('test 3')

In [2]:
import time